In [2]:
import os
import re
from openai import OpenAI
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility, MilvusClient

# OpenAI API 클라이언트 설정
client = OpenAI()
client.organization = 'org-bViUymTUSW9oQ1iJBmC2xi8M'

In [3]:

# Milvus에 연결 설정 (로컬에서 실행 중인 Milvus에 연결)
def connect_to_milvus():
    try:
        connections.connect(alias="default", host='127.0.0.1', port='19530')
        print("Milvus에 성공적으로 연결되었습니다.")
    except Exception as e:
        print(f"Milvus 연결 실패: {str(e)}")

# 불용어 목록 (예시)
stopwords = set(["이", "그", "저", "에서", "으로", "에", "를", "이", "는", "가", "과", "을"])

def remove_stopwords(text):
    # 불용어 제거 후 토큰화
    words = re.findall(r'\w+', text)  # 간단한 토큰화 (단어 추출)
    filtered_words = [word for word in words if word not in stopwords]
    return filtered_words

In [4]:
def process_and_tokenize_file(file_path):
    # 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 불용어 제거 및 토큰화
    tokens = remove_stopwords(content)
    
    # 토큰화된 내용을 파일로 저장
    tokenized_file_path = file_path.replace(".txt", "_tokenized.txt")
    with open(tokenized_file_path, 'w', encoding='utf-8') as tokenized_file:
        tokenized_file.write(" ".join(tokens))
    
    return tokens, tokenized_file_path

def generate_embeddings(text_tokens):
    # OpenAI API를 사용하여 임베딩 생성
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=" ".join(text_tokens)
    )
    return response['data'][0]['embedding']

In [5]:
def create_milvus_collection(collection_name):
    # Milvus에서 컬렉션이 있는지 확인하고, 없으면 생성
    if not utility.has_collection(collection_name):
        # 컬렉션 스키마 정의
        fields = [
            FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=1536),  # OpenAI 임베딩 크기 1536
            FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512)
        ]
        schema = CollectionSchema(fields=fields, description="Text embeddings for chapters")
        collection = Collection(name=collection_name, schema=schema)
        print(f"Milvus 컬렉션 '{collection_name}'이 생성되었습니다.")
    else:
        collection = Collection(collection_name)
        print(f"Milvus 컬렉션 '{collection_name}'이 이미 존재합니다.")
    
    return collection

def insert_into_milvus(collection, text, embedding):
    # Milvus에 임베딩과 텍스트를 삽입
    collection.insert([embedding, text])
    collection.flush()  # 데이터를 Milvus에 저장
    print(f"임베딩과 텍스트가 '{collection.name}' 컬렉션에 저장되었습니다.")

In [6]:

def main():
    # Milvus 연결
    connect_to_milvus()

    # 1gapja.txt 파일 처리
    file_path = "1gapja.txt"
    tokens, tokenized_file_path = process_and_tokenize_file(file_path)
    
    # 임베딩 생성
    print("임베딩 생성 중...")
    embedding = generate_embeddings(tokens)
    
    # 파일명에서 챕터명을 추출
    chapter_name = "1gapja"  # 챕터 이름은 파일명에서 추출된다고 가정
    
    # Milvus 컬렉션 생성 및 저장
    collection = create_milvus_collection(chapter_name)
    insert_into_milvus(collection, " ".join(tokens), embedding)
    print(f"처리 완료: {file_path}")

if __name__ == "__main__":
    main()

Milvus 연결 실패: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:19530, illegal connection params or server unavailable)>
임베딩 생성 중...


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 205687 tokens (205687 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}